# Unrestricted Open-Shell Hartree-Fock

In the first two tutorials in this module, we wrote programs which implement a closed-shell formulation of Hartree-Fock theory using restricted orbitals, aptly named Restricted Hartree-Fock (RHF).  In this tutorial, we will abandon strictly closed-shell systems and the notion of restricted orbitals, in favor of a more general theory known as Unrestricted Hartree-Fock (UHF) which can accommodate more diverse molecules.  In UHF, the orbitals occupied by spin up ($\alpha$) electrons and those occupied by spin down ($\beta$) electrons no longer have the same spatial component, e.g., 

$$\chi_i({\bf x}) = \begin{cases}\psi^{\alpha}_j({\bf r})\alpha(\omega) \\ \psi^{\beta}_j({\bf r})\beta(\omega)\end{cases},$$

meaning that they will not have the same orbital energy.  This relaxation of orbital constraints allows for more variational flexibility, which leads to UHF always being able to find a lower total energy solution than RHF.  

## I. Theoretical Overview
In UHF, we seek to solve the coupled equations

\begin{align}
{\bf F}^{\alpha}{\bf C}^{\alpha} &= {\bf SC}^{\alpha}{\bf\epsilon}^{\alpha} \\
{\bf F}^{\beta}{\bf C}^{\beta} &= {\bf SC}^{\beta}{\bf\epsilon}^{\beta},
\end{align}

which are the unrestricted generalizations of the restricted Roothan equations, called the Pople-Nesbitt equations.  Here, the one-electron Fock matrices are given by

\begin{align}
F_{\mu\nu}^{\alpha} &= H_{\mu\nu} + (\mu\,\nu\mid\lambda\,\sigma)[D_{\lambda\sigma}^{\alpha} + D_{\lambda\sigma}^{\beta}] - (\mu\,\lambda\,\mid\nu\,\sigma)D_{\lambda\sigma}^{\beta}\\
F_{\mu\nu}^{\beta} &= H_{\mu\nu} + (\mu\,\nu\mid\,\lambda\,\sigma)[D_{\lambda\sigma}^{\alpha} + D_{\lambda\sigma}^{\beta}] - (\mu\,\lambda\,\mid\nu\,\sigma)D_{\lambda\sigma}^{\alpha},
\end{align}

where the density matrices $D_{\lambda\sigma}^{\alpha}$ and $D_{\lambda\sigma}^{\beta}$ are given by

\begin{align}
D_{\lambda\sigma}^{\alpha} &= C_{\sigma i}^{\alpha}C_{\lambda i}^{\alpha}\\
D_{\lambda\sigma}^{\beta} &= C_{\sigma i}^{\beta}C_{\lambda i}^{\beta}.
\end{align}

Unlike for RHF, the orbital coefficient matrices ${\bf C}^{\alpha}$ and ${\bf C}^{\beta}$ are of dimension $M\times N^{\alpha}$ and $M\times N^{\beta}$, where $M$ is the number of AO basis functions and $N^{\alpha}$ ($N^{\beta}$) is the number of $\alpha$ ($\beta$) electrons.  The total UHF energy is given by

\begin{align}
E^{\rm UHF}_{\rm total} &= E^{\rm UHF}_{\rm elec} + E^{\rm BO}_{\rm nuc},\;\;{\rm with}\\
E^{\rm UHF}_{\rm elec} &= \frac{1}{2}[({\bf D}^{\alpha} + {\bf D}^{\beta}){\bf H} + 
{\bf D}^{\alpha}{\bf F}^{\alpha} + {\bf D}^{\beta}{\bf F}^{\beta}].
\end{align}

## II. Implementation

In any SCF program, there will be several common elements which can be abstracted from the program itself into separate modules, classes, or functions to 'clean up' the code that will need to be written explicitly; examples of this concept can be seen throughout the Psi4NumPy reference implementations.  For the purposes of this tutorial, we can achieve some degree of code cleanup without sacrificing readabilitiy and clarity by focusing on abstracting only the parts of the code which are both 
- Lengthy subroutines, and 
- Used repeatedly.  

In our UHF program, let's use what we've learned in the last tutorial by also implementing DIIS convergence accelleration for our SCF iterations.  With this in mind, two subroutines in particular would benefit from abstraction are

1. Orthogonalize & diagonalize Fock matrix
2. Extrapolate previous trial vectors for new DIIS solution vector

Before we start writing our UHF program, let's try to write functions which can perform the above tasks so that we can use them in our implementation of UHF.  Recall that defining functions in Python has the following syntax:
~~~python
def function_name(*args **kwargs):
    # function block
    return return_values
~~~
A thorough discussion of defining functions in Python can be found [here](https://docs.python.org/2/tutorial/controlflow.html#defining-functions "Go to Python docs").  First, let's write a function which can diagonalize the Fock matrix and return the orbital coefficient matrix **C** and the density matrix **D**.  From our RHF tutorial, this subroutine is executed with:
~~~python
F_p =  A.dot(F).dot(A)
e, C_p = np.linalg.eigh(F_p)
C = A.dot(C_p)
C_occ = C[:, :ndocc]
D = np.einsum('pi,qi->pq', C_occ, C_occ)
~~~
Examining this code block, there are three quantities which must be specified beforehand:
- Fock matrix, **F**
- Orthogonalization matrix, ${\bf A} = {\bf S}^{-1/2}$
- Number of doubly occupied orbitals, `ndocc`

However, since the orthogonalization matrix **A** is a static quantity (only built once, then left alone) we may choose to leave **A** as a *global* quantity, instead of an argument to our function.  In the cell below, using the code snippet given above, write a function `diag_F()` which takes **F** and the number of orbitals `norb` as arguments, and returns **C** and **D**:

In [1]:
# ==> Define function to diagonalize F <==
def diag_F(F, norb):
    F_p = A.dot(F).dot(A)
    e, C_p = np.linalg.eigh(F_p)
    C = A.dot(C_p)
    C_occ = C[:, :norb]
    D = np.einsum('pi,qi->pq', C_occ, C_occ)
    return (C, D)

Next, let's write a function to perform DIIS extrapolation and generate a new solution vector.  Recall that the DIIS-accellerated SCF algorithm is:
#### Algorithm 1: DIIS within a generic SCF Iteration
1. Compute **F**, append to list of previous trial vectors
2. Compute AO orbital gradient **r**, append to list of previous residual vectors
3. Compute RHF energy
3. Check convergence criteria
    - If RMSD of **r** sufficiently small, and
    - If change in SCF energy sufficiently small, break
4. Build **B** matrix from previous AO gradient vectors
5. Solve Pulay equation for coefficients $\{c_i\}$
6. Compute DIIS solution vector **F_DIIS** from $\{c_i\}$ and previous trial vectors
7. Compute new orbital guess with **F_DIIS**

In our function, we will perform steps 4-6 of the above algorithm.  What information will we need to provide our function in order to do so?  To build **B** (step 4 above) in the previous tutorial, we used:
~~~python
# Build B matrix
B_dim = len(F_list) + 1
B = np.empty((B_dim, B_dim))
B[-1, :] = -1
B[:, -1] = -1
B[-1, -1] = 0
for i in xrange(len(F_list)):
    for j in xrange(len(F_list)):
        B[i, j] = np.einsum('ij,ij->', DIIS_RESID[i], DIIS_RESID[j])
~~~
Here, we see that we must have all previous DIIS residual vectors (`DIIS_RESID`), as well as knowledge about how many previous trial vectors there are (for the dimension of **B**).  To solve the Pulay equation (step 5 above):
~~~python
# Build RHS of Pulay equation 
rhs = np.zeros((B_dim))
rhs[-1] = -1
      
# Solve Pulay equation for c_i's with NumPy
coeff = np.linalg.solve(B, rhs)
~~~
For this step, we only need the dimension of **B** (which we computed in step 4 above) and a NumPy routine, so this step doesn't require any additional arguments.  Finally, to build the DIIS Fock matrix (step 6):
~~~python
# Build DIIS Fock matrix
F = np.zeros_like(F_list[0])
for x in xrange(coeff.shape[0] - 1):
    F += coeff[x] * F_list[x]
~~~
Clearly, for this step, we need to know all the previous trial vectors (`F_list`) and the coefficients we generated in the previous step.  In the cell below, write a funciton `diis_xtrap()` according to Algorithm 1 steps 4-6, using the above code snippets, which takes a list of previous trial vectors `F_list` and residual vectors `DIIS_RESID` as arguments and returns the new DIIS solution vector `F_DIIS`:

In [13]:
# ==> Build DIIS Extrapolation Function <==
def diis_xtrap(F_list, DIIS_RESID):
    # Build B matrix
    B_dim = len(F_list) + 1
    B = np.empty((B_dim, B_dim))
    B[-1, :] = -1
    B[:, -1] = -1
    B[-1, -1] = 0
    for i in range(len(F_list)):
        for j in range(len(F_list)):
            B[i, j] = np.einsum('ij,ij->', DIIS_RESID[i], DIIS_RESID[j])

    # Build RHS of Pulay equation 
    rhs = np.zeros((B_dim))
    rhs[-1] = -1
      
    # Solve Pulay equation for c_i's with NumPy
    coeff = np.linalg.solve(B, rhs)
      
    # Build DIIS Fock matrix
    F_DIIS = np.zeros_like(F_list[0])
    for x in range(coeff.shape[0] - 1):
        F_DIIS += coeff[x] * F_list[x]
    
    return F_DIIS

We are now ready to begin writing our UHF program!  Let's begin by importing <span style='font-variant: small-caps'> Psi4 </span> and NumPy, and defining our molecule & basic options:

In [3]:
# ==> Import Psi4 & NumPy <==
import psi4
import numpy as np

In [4]:
# ==> Set Basic Psi4 Options <==
# Memory specification
psi4.set_memory(int(5e8))
numpy_memory = 2

# Set output file
psi4.core.set_output_file('output.dat', False)

# Define Physicist's water -- don't forget C1 symmetry!
mol = psi4.geometry("""
O
H 1 1.1
H 1 1.1 2 104
symmetry c1
""")

# Set computation options
psi4.set_options({'guess': 'core',
                  'basis': 'cc-pvdz',
                  'scf_type': 'pk',
                  'e_convergence': 1e-8,
                  'reference': 'uhf'})

You may notice that in the above `psi4.set_options()` block, there are two additional options -- namely, `'guess': 'core'` and `'reference': 'uhf'`.  These options make sure that when we ultimately check our program against <span style='font-variant: small-caps'> Psi4</span>, the options <span style='font-variant: small-caps'> Psi4 </span> uses are identical to our implementation.  Next, let's define the options for our UHF program; we can borrow these options from our RHF implementation with DIIS accelleration that we completed in our last tutorial.

In [5]:
# ==> Set default program options <==
# Maximum SCF iterations
MAXITER = 40
# Energy convergence criterion
E_conv = 1.0e-6
D_conv = 1.0e-3

Static quantities like the ERI tensor, core Hamiltonian, and orthogonalization matrix have exactly the same form in UHF as in RHF.  Unlike in RHF, however, we will need the number of $\alpha$ and $\beta$ electrons.  Fortunately, both these values are available through querying the Wavefunction object.  In the cell below, generate these static objects and compute each of the following:
- Number of basis functions, `nbf`
- Number of alpha electrons, `nalpha`
- Number of beta electrons, `nbeta`
- Number of doubly occupied orbitals, `ndocc` (Hint: In UHF, there can be unpaired electrons!)

In [6]:
# ==> Compute static 1e- and 2e- quantities with Psi4 <==
# Class instantiation
wfn = psi4.core.Wavefunction.build(mol, psi4.core.get_global_option('basis'))
mints = psi4.core.MintsHelper(wfn.basisset())

# Overlap matrix
S = np.asarray(mints.ao_overlap())

# Number of basis Functions, alpha & beta orbitals, and # doubly occupied orbitals
nbf = wfn.nso()
nalpha = wfn.nalpha()
nbeta = wfn.nbeta()
ndocc = min(nalpha, nbeta)

print('Number of basis functions: %d' % (nbf))
print('Number of singly occupied orbitals: %d' % (abs(nalpha - nbeta)))
print('Number of doubly occupied orbitals: %d' % (ndocc))

# Memory check for ERI tensor
I_size = (nbf**4) * 8.e-9
print('\nSize of the ERI tensor will be {:4.2f} GB.'.format(I_size))
memory_footprint = I_size * 1.5
if I_size > numpy_memory:
    psi4.core.clean()
    raise Exception("Estimated memory utilization (%4.2f GB) exceeds allotted memory \
                     limit of %4.2f GB." % (memory_footprint, numpy_memory))

# Build ERI Tensor
I = np.asarray(mints.ao_eri())

# Build core Hamiltonian
T = np.asarray(mints.ao_kinetic())
V = np.asarray(mints.ao_potential())
H = T + V

# Construct AO orthogonalization matrix A
A = mints.ao_overlap()
A.power(-0.5, 1.e-16)
A = np.asarray(A)

Number of basis functions: 24
Number of singly occupied orbitals: 0
Number of doubly occupied orbitals: 5

Size of the ERI tensor will be 0.00 GB.


Unlike the static quantities above, the CORE guess in UHF is slightly different than in RHF.  Since the $\alpha$ and $\beta$ electrons do not share spatial orbitals, we must construct a guess for *each* of the $\alpha$ and $\beta$ orbitals and densities.  In the cell below, using the function `diag_F()`, construct the CORE guesses and compute the nuclear repulsion energy:

(Hint: The number of $\alpha$ orbitals is the same as the number of $\alpha$ electrons!)

In [7]:
# ==> Build alpha & beta CORE guess <==
Ca, Da = diag_F(H, nalpha)
Cb, Db = diag_F(H, nbeta)

# Get nuclear repulsion energy
E_nuc = mol.nuclear_repulsion_energy()

We are almost ready to perform our SCF iterations; beforehand, however, we must initiate variables for the current & previous SCF energies, and the lists to hold previous residual vectors and trial vectors for the DIIS procedure.  Since, in UHF, there are Fock matrices ${\bf F}^{\alpha}$ and ${\bf F}^{\beta}$ for both $\alpha$ and $\beta$ orbitals, we must apply DIIS to each of these matrices separately.  In the cell below, define empty lists to hold previous Fock matrices and residual vectors for both $\alpha$ and $\beta$ orbitals:

In [8]:
# ==> Pre-Iteration Setup <==
# SCF & Previous Energy
SCF_E = 0.0
E_old = 0.0

We are now ready to write the SCF iterations.  The algorithm for UHF-SCF iteration, with DIIS convergence accelleration, is:
#### Algorithm 2: DIIS within UHF-SCF Iteration
1. Build ${\bf F}^{\alpha}$ and ${\bf F}^{\beta}$, append to trial vector lists
2. Compute the DIIS residual for $\alpha$ and $\beta$, append to residual vector lists
3. Compute UHF energy
4. Convergence check
    - If average of RMSD of $\alpha$ and $\beta$ residual sufficiently small, and
    - If change in UHF energy sufficiently small, break
5. DIIS extrapolation of ${\bf F}^{\alpha}$ and ${\bf F}^{\beta}$ to form new solution vector
6. Compute new ${\alpha}$ and ${\beta}$ orbital & density guesses

In the cell below, write the UHF-SCF iteration according to Algorithm 2:

(Hint: Use your functions `diis_xtrap()` and `diag_F` for Algorithm 2 steps 5 & 6, respectively)

In [14]:
# Trial & Residual Vector Lists -- one each for alpha & beta
F_list_a = []
F_list_b = []
R_list_a = []
R_list_b = []

# ==> UHF-SCF Iterations <==
print('==> Starting SCF Iterations <==\n')

# Begin Iterations
for scf_iter in range(MAXITER):
    # Build Fa & Fb matrices
    Ja = np.einsum('pqrs,rs->pq', I, Da)
    Jb = np.einsum('pqrs,rs->pq', I, Db)
    Ka = np.einsum('prqs,rs->pq', I, Da)
    Kb = np.einsum('prqs,rs->pq', I, Db)
    Fa = H + (Ja + Jb) - Ka
    Fb = H + (Ja + Jb) - Kb
    
    # Compute DIIS residual for Fa & Fb
    diis_r_a = A.dot(Fa.dot(Da).dot(S) - S.dot(Da).dot(Fa)).dot(A)
    diis_r_b = A.dot(Fb.dot(Db).dot(S) - S.dot(Db).dot(Fb)).dot(A)
    
    # Append trial & residual vectors to lists
    F_list_a.append(Fa)
    F_list_b.append(Fb)
    R_list_a.append(diis_r_a)
    R_list_b.append(diis_r_b)
    
    # Compute UHF Energy
    SCF_E = np.einsum('pq,pq->', (Da + Db), H)
    SCF_E += np.einsum('pq,pq->', Da, Fa)
    SCF_E += np.einsum('pq,pq->', Db, Fb)
    SCF_E *= 0.5
    SCF_E += E_nuc
    
    dE = SCF_E - E_old
    dRMS = 0.5 * (np.mean(diis_r_a**2)**0.5 + np.mean(diis_r_b**2)**0.5)
    print('SCF Iteration %3d: Energy = %4.16f dE = % 1.5E dRMS = %1.5E' % (scf_iter, SCF_E, dE, dRMS))
    
    # Convergence Check
    if (abs(dE) < E_conv) and (dRMS < D_conv):
        break
    E_old = SCF_E
    
    # DIIS Extrapolation
    if scf_iter >= 2:
        Fa = diis_xtrap(F_list_a, R_list_a)
        Fb = diis_xtrap(F_list_b, R_list_b)
    
    # Compute new orbital guess
    Ca, Da = diag_F(Fa, nalpha)
    Cb, Db = diag_F(Fb, nbeta)
    
    # MAXITER exceeded?
    if (scf_iter == MAXITER):
        psi4.core.clean()
        raise Exception("Maximum number of SCF iterations exceeded.")

# Post iterations
print('\nSCF converged.')
print('Final UHF Energy: %.8f [Eh]' % SCF_E)

==> Starting SCF Iterations <==

SCF Iteration   0: Energy = -74.1207806468836452 dE =  0.00000E+00 dRMS = 8.64677E-02
SCF Iteration   1: Energy = -74.8671819457688485 dE = -7.46401E-01 dRMS = 6.52840E-02
SCF Iteration   2: Energy = -75.4149087803903342 dE = -5.47727E-01 dRMS = 5.21690E-02
SCF Iteration   3: Energy = -75.9800488561561309 dE = -5.65140E-01 dRMS = 6.34267E-03
SCF Iteration   4: Energy = -75.9894383301614340 dE = -9.38947E-03 dRMS = 5.45826E-04
SCF Iteration   5: Energy = -75.9897683674259383 dE = -3.30037E-04 dRMS = 1.70671E-04
SCF Iteration   6: Energy = -75.9897948623176376 dE = -2.64949E-05 dRMS = 4.28126E-05
SCF Iteration   7: Energy = -75.9897957712875609 dE = -9.08970E-07 dRMS = 5.40285E-06

SCF converged.
Final UHF Energy: -75.98979577 [Eh]


Congratulations! You've written your very own Unrestricted Hartree-Fock program with DIIS convergence accelleration!  Finally, let's check your final UHF energy against <span style='font-variant: small-caps'> Psi4</span>:

In [15]:
# Compare to Psi4
SCF_E_psi = psi4.energy('SCF')
psi4.compare_values(SCF_E_psi, SCF_E, 6, 'SCF Energy')

	SCF Energy........................................................PASSED


## References
1. A. Szabo and N. S. Ostlund, *Modern Quantum Chemistry*, Introduction to Advanced Electronic Structure Theory. Courier Corporation, 1996.
2. I. N. Levine, *Quantum Chemistry*. Prentice-Hall, New Jersey, 5th edition, 2000.
3. T. Helgaker, P. Jorgensen, and J. Olsen, *Molecular Electronic Structure Theory*, John Wiley & Sons Inc, 2000.